In [1]:
import ngsolve as ngs
from do import solve, solve_2d
from ngsolve.webgui import Draw
import matplotlib.pyplot as plt
from copy import deepcopy
import netgen.occ as occ

levels = 5
deg = 2
degdif = 1

######################################################
# from bhdata import BHCurves
# fun_dw  = BHCurves(4)
# fun_w   = fun_dw.Integrate()
# fun_ddw = fun_dw.Differentiate()
######################################################
from bhdata import Brauer, BrauerCut
fun_w, fun_dw, fun_ddw  = Brauer()
######################################################

mu0 = 1.256636e-6
nu0 = 1/mu0

linear = "coil_plus|coil_minus"
nonlinear = "stator"

# linear = "stator|coil_plus|coil_minus"
# nonlinear = ""

strom = 1e5*2
amps = 0.025**2*3.15*strom
print(f'Applying a current of {amps:.2f} Amps')
######################################################

r_coil = 0.025
y_coil = 0.05
r_outer = 0.1

c1 = occ.WorkPlane().Circle(r_outer).Face()
c2 = occ.WorkPlane().MoveTo(0,y_coil).Circle(r_coil).Face()
c3 = occ.WorkPlane().MoveTo(0,-y_coil).Circle(r_coil).Face()

full = occ.Glue([c1,c2,c3])

full.faces[0].name = 'stator'
full.faces[1].name = 'coil_plus'
full.faces[2].name = 'coil_minus'

full.edges[0].name = 'outer'

geoOCC = occ.OCCGeometry(full, dim = 2)
Draw(full)

Applying a current of 393.75 Amps


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': …

BaseWebGuiScene

In [2]:
meshes = []; its = []; errors = []

for i in range(levels):
    ngmesh = geoOCC.GenerateMesh()
    mesh = ngs.Mesh(ngmesh)

    for j in range(i):
        mesh.Refine()
    mesh.Curve(6)

    with ngs.TaskManager(): J = mesh.MaterialCF({'coil_plus': strom, 'coil_minus': -strom, 'stator': 0}, default = 0)
    H1_0 = ngs.H1(mesh, order = deg, dirichlet = "outer")
    L2_0 = ngs.L2(mesh, dim = 2, order = deg-1)
    A0 = ngs.GridFunction(H1_0)
    A0, it = solve_2d(H1_0, A0, mesh, deg, J, fun_w, fun_dw, fun_ddw, linear, nonlinear)
    its.append(it)
    B0 = ngs.GridFunction(L2_0, nested = True)
    with ngs.TaskManager(): B0.Set(ngs.grad(A0), bonus_intorder = 10)



    H1_1 = ngs.H1(mesh, order = deg+degdif, dirichlet = "outer")
    L2_1 = ngs.L2(mesh, dim = 2, order = deg+degdif-1)
    A1 = ngs.GridFunction(H1_1)
    A1, it = solve_2d(H1_1, A1, mesh, deg+degdif, J, fun_w, fun_dw, fun_ddw, linear, nonlinear)
    its.append(it)
    B1 = ngs.GridFunction(L2_1, nested = True)
    with ngs.TaskManager(): B1.Set(ngs.grad(A1), bonus_intorder = 10)

    with ngs.TaskManager(): error = ngs.Integrate((B0-B1)**2, mesh, order = 3*(deg+degdif))**(1/2)/\
                                    ngs.Integrate((B1)**2, mesh, order = 3*(deg+degdif))**(1/2)
    print(error)
    errors.append(error)
    
print(its)

Using 2D mesh with ne= 191 elements and nv= 107 points and  2441 DOFs.
 
Iter:  1 | assem : 0.02s | CG took 0.01s | alpha : 0.50 | energy = 0.0000000762 | relres = 1.00e+00 |
Iter:  2 | assem : 0.02s | CG took 0.01s | alpha : 1.00 | energy = -23.9211956525 | relres = 5.28e-01 |
Iter:  3 | assem : 0.01s | CG took 0.01s | alpha : 0.50 | energy = -24.7593095204 | relres = 1.99e-01 |
Iter:  4 | assem : 0.01s | CG took 0.01s | alpha : 0.12 | energy = -24.8882121679 | relres = 1.20e-01 |
Iter:  5 | assem : 0.01s | CG took 0.01s | alpha : 0.06 | energy = -24.9038603207 | relres = 1.06e-01 |
Iter:  6 | assem : 0.01s | CG took 0.01s | alpha : 0.02 | energy = -24.9094375330 | relres = 9.95e-02 |
Iter:  7 | assem : 0.01s | CG took 0.01s | alpha : 0.00 | energy = -24.9106355217 | relres = 9.80e-02 |
Iter:  8 | assem : 0.01s | CG took 0.01s | alpha : 0.00 | energy = -24.9109230690 | relres = 9.76e-02 |
Iter:  9 | assem : 0.06s | CG took 0.01s | alpha : 0.00 | energy = -24.9109230690 | relres = 9.76

In [3]:
# meshes = []; Bs = []; Bps = []; its = []; errors = []

# for i in range(levels):
#     ngmesh = geoOCC.GenerateMesh()
#     mesh = ngs.Mesh(ngmesh)

#     for j in range(i):
#         mesh.Refine()
#     mesh.Curve(6)

#     with ngs.TaskManager(): J = mesh.MaterialCF({'coil_plus': strom, 'coil_minus': -strom, 'stator': 0}, default = 0)
#     H1 = ngs.H1(mesh, order = deg, dirichlet = "outer", autoupdate = True)
#     L2 = ngs.L2(mesh, dim = 2, order = deg-1)
#     # L2 = ngs.HDiv(mesh, order = deg-1)
#     A0 = ngs.GridFunction(H1)
#     A0, it = solve_2d(H1, A0, mesh, deg, J, fun_w, fun_dw, fun_ddw, linear, nonlinear)
#     its.append(it)

#     B = ngs.GridFunction(L2, nested = True)
#     with ngs.TaskManager(): B.Set(ngs.grad(A0), bonus_intorder = 10)
#     Bs.append(B)

#     # Draw(B, mesh)

#     if i>0:
#         L2 = ngs.L2(mesh, dim = 2, order = deg-1)
#         # L2 = ngs.HDiv(mesh, order = deg-1)
#         Bp = ngs.GridFunction(L2)

#         u,v = L2.TnT()
#         ir = ngs.IntegrationRule(ngs.fem.ET.TRIG, order = 3*deg)

#         # with ngs.TaskManager():
#         #     a = ngs.BilinearForm(L2)
#         #     a += ngs.SymbolicBFI(u*v, intrule = ir); a.Assemble()
#         #     f = ngs.LinearForm(L2)
#         #     f += ngs.SymbolicLFI(Bs[i-1]*v, intrule = ir); f.Assemble()
#         #     Bp.vec.data = a.mat.Inverse(freedofs = L2.FreeDofs())*f.vec

#         with ngs.TaskManager(): Bp.Set(Bs[i-1], bonus_intorder = 10)

#         Bps.append(Bp)
#         # Draw(Bp, mesh)
#         with ngs.TaskManager(): error = ngs.Integrate((B-Bp)**2, mesh, order = 3*deg)**(1/2)/\
#                                         ngs.Integrate((B)**2, mesh, order = 3*deg)**(1/2)
#         print(error)
#         errors.append(error)

# print(its)

In [4]:
import numpy as np
a = np.array(errors)
# a = np.array([0.00011907968926321385,7.813336806776809e-06
# , 3.0480923924668197e-07
# ,1.0782162011697921e-08
# ])
# print(errors)
# print(a[:-1]/a[1:])
print(np.log2(a[:-1]/a[1:]))

# [1.86846328 1.99313412 2.08303115 2.32839828] # gesetz 4
# [1.90055911 2.02087594 2.02998492 2.31871363] # gesetz 3

# [2.36471921 3.7432967  3.82449012 3.90597432]

# [3.42757099 3.72076194 3.9574683  3.96120833] # deg 4
# [4.29694018 4.67219892 4.92504455 3.55727334] # deg 5, tol2 = 1e-8
# [4.2969388  4.67219923 4.92497567 4.5799986 ] # deg 5, tol2 = 1e-12
# [4.29694018 4.67219938 4.92444852 4.98808301] # deg 5, tol2 = 1e-15

[4.29694018 4.67219938 4.92444852 4.98808301]


In [5]:
with ngs.TaskManager(): Draw(B0, mesh)
with ngs.TaskManager(): Draw(B1, mesh)
with ngs.TaskManager(): Draw(ngs.log(ngs.Norm(B0-B1)), mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [6]:
# help(ngs.GridFunction.Set)

In [7]:
ngs.exp

<function ngsolve.fem.PyCapsule.exp>